In [1]:
import pandas as pd
import numpy as np
import os
import io
import geopandas as gpd
import zipfile
import requests
import fiona
import folium
import branca
from sklearn.neighbors import KDTree
from sklearn.neighbors import BallTree


In [2]:
# urls and definitions

## kerncijfers buurten en wijken
bw_cijfers_url = [
    'http://download.cbs.nl/regionale-kaarten/kwb-2004-versie-2011-08-16.xls',
    'http://download.cbs.nl/regionale-kaarten/kwb-2005-versie-2011-08-16.xls',
    'http://download.cbs.nl/regionale-kaarten/kwb-2006-versie-2011-08-16.xls',
    'http://download.cbs.nl/regionale-kaarten/kwb-2007-versie-2011-08-16.xls',
    'http://download.cbs.nl/regionale-kaarten/kwb-2008-versie-2011-11-23.xls',
    'http://download.cbs.nl/regionale-kaarten/kwb-2009-versie-2014-06-01.xls',
    'http://download.cbs.nl/regionale-kaarten/kwb-2010-versie-2013-12.xls',
    'http://download.cbs.nl/regionale-kaarten/kwb-2011.xls',
    'http://download.cbs.nl/regionale-kaarten/kwb-2012.xls',
    'http://download.cbs.nl/regionale-kaarten/kwb-2013.xls'
    # 'http://download.cbs.nl/regionale-kaarten/kwb-2014.xls',
    # 'https://www.cbs.nl/-/media/cbs/dossiers/nederland-regionaal/wijk-en-buurtstatistieken/_exel/kwb-2015.xls',
    # 'https://www.cbs.nl/-/media/cbs/dossiers/nederland-regionaal/wijk-en-buurtstatistieken/_exel/kwb-2016.xls',
    # 'https://www.cbs.nl/-/media/cbs/dossiers/nederland-regionaal/wijk-en-buurtstatistieken/_exel/kwb-2017.xls',
    # 'https://www.cbs.nl/-/media/cbs/dossiers/nederland-regionaal/wijk-en-buurtstatistieken/_exel/kwb-2018.xls'
]

## buurt and wijk kaarten
bw_kaarten_url = [
    ['https://www.cbs.nl/-/media/imported/documents/2008/01/2004-buurtkaart-data.zip?la=nl-nl'],
    ['https://www.cbs.nl/-/media/imported/documents/2008/01/2005-buurtkaart-data1-v2.zip?la=nl-nl',
     'https://www.cbs.nl/-/media/imported/documents/2010/13/2005buurtkaartdata2v2.zip?la=nl-nl'],
    ['https://www.cbs.nl/-/media/imported/documents/2010/13/buurt_2006_gn2.zip?la=nl-nl'],
    ['https://www.cbs.nl/-/media/imported/documents/2009/01/buurt_2007_gen(2).zip?la=nl-nl'],
    ['http://download.cbs.nl/regionale-kaarten/2008-buurtkaart-gn-3.zip'],
    ['http://download.cbs.nl/regionale-kaarten/2009-buurtkaart-gn-3.zip'],
    ['http://download.cbs.nl/regionale-kaarten/2010-buurtkaart-shape-versie-3.zip'],
    ['http://download.cbs.nl/regionale-kaarten/2011-buurtkaart-shape-versie-3.zip'],
    ['http://download.cbs.nl/regionale-kaarten/shape-2012-versie-3.0.zip'],
    ['http://download.cbs.nl/regionale-kaarten/shape-2013-versie-3-0.zip']
    # ['https://www.cbs.nl/-/media/_pdf/2016/35/shape-2014-versie-30.zip'],
    # ['https://www.cbs.nl/-/media/_pdf/2017/36/buurt_2015.zip'],
    # ['https://www.cbs.nl/-/media/cbs/dossiers/nederland-regionaal/wijk-en-buurtstatistieken/shape-2016-versie-30.zip'],
    # ['https://www.cbs.nl/-/media/cbs/dossiers/nederland-regionaal/wijk-en-buurtstatistieken/wijkbuurtkaart_2017_v3.zip'],
    # ['https://www.cbs.nl/-/media/cbs/dossiers/nederland-regionaal/wijk-en-buurtstatistieken/wijkbuurtkaart_2018_v3.zip']
]

# definitions
years = np.arange(2004, 2014)

output_path = '/home/trond/vinex/data/'


In [92]:
# functions

def find_columns(dat):
    # keep only the following columns
    col_list = ['OAD']
    for name in dat.columns:
        if any(item in name for item in col_list):
            out = name
    
    return out

def create_gwb_code(code):
    code = str(code)
    if 'BU' in code:
        return code
    else:
        return 'BU' + str(0)*(8-len(code)) + code

def read_and_define_subsample(url):
    version = [year for year in years if str(year) in url][0]
    print('Now downloading version ' + str(version))
    # Download data
    try:
        dat = pd.read_excel(url)
        # Convert the header names to all uppercase
        dat.columns = dat.columns.str.upper()
        # find col_list
        cols = [name for name in dat.columns[:1]]
        cols.append(find_columns(dat))
        # buurt variants
        buurt_list = ['BUURT', 'Buurt', 'buurt', 'B']
        # Subset to buurt observations and relevant columns
        dat = dat.loc[dat['RECS'].apply(lambda x: x in buurt_list), cols]
        # fix columns
        dat.columns = ['statcode', 'OAD']
        dat['year'] = version
        # correct the buurt wijk identifier
        dat['statcode'] = dat['statcode'].apply(lambda x: create_gwb_code(x))
    except Exception as e:
        print('An error occurred: ' + str(e))
    
    return dat

def download_buurt_shp_from_url(url, local_path):
    # Download data and return list of geofiles
    try:
        # find content from url
        r = requests.get(url)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        # select only buurt level files
        buurt_files = [f for f in sorted(z.namelist()) for brt in ['buurt', 'brt'] if brt in f]
        # select the geofiles
        geo_files = [y for y in buurt_files for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)]
        for g in geo_files:
            z.extract(g, local_path)
    except Exception as e:
        print('An error ocurred: ' + str(e))

    return geo_files

def read_buurt_shp(url):
    # find version
    version = [year for year in years if str(year) in url[0]][0]
    print('Now downloading version ' + str(version))
    # create a new temporary directory
    tmp_path = output_path + 'tmp/'
    if not os.path.exists(tmp_path):
        os.makedirs(tmp_path)
    # find all geofiles    
    geo_files = []
    for u in url:
        geo_files = geo_files + download_buurt_shp_from_url(u, tmp_path)
    print(geo_files)
    # read shape file
    shp_file = [f for f in geo_files if f.endswith('.shp')] 
    dat = gpd.read_file(tmp_path + shp_file[0])
    # find col_list
    cols = [name for name in dat.columns[:1]]
    cols.append(find_columns(dat))
    # clean up
    for g in geo_files:
        os.remove(tmp_path + g)
    for d in os.listdir(tmp_path):
        os.rmdir(tmp_path + d)
        
    return dat.loc[:,cols + ['geometry']]

def read_and_extract_zip(url):
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(output_path + 'gebiedsindelingen')

## definition and function for creating map

## from https://geospatialawarenesshub.com/blog/make-interactive-choropleth-maps/
def create_folium_map(gdf, n_cats, target_var, display_vars, yx_point):
    # make sure the index is correct
    gdf['idx'] = np.arange(0,len(gdf))
    gdf = gdf.set_index('idx') 
    # base map
    m = folium.Map(location=yx_point, zoom_start=10, tiles='cartodbpositron')
    # create color palette
    colormap = branca.colormap.linear.RdYlBu_11.to_step(n_cats).scale(0, n_cats-1)
    # create dictionary for styling
    v_dict = gdf[target_var]
    # style function for geojson
    def style_function(feature):
        v_d = v_dict.get(int(feature['id']), None)
        return {
            'fillColor': '#black' if v_d is None else colormap(v_d),
            'color': 'black',
            'weight': 0.2,
            'dashArray': '5, 5',
            'fillOpacity': 0.6,
        }
    x=folium.GeoJson(
        gdf,
        name= target_var,
        style_function=style_function,
        highlight_function= None,
        tooltip=(gdf[i] for i in gdf[display_vars])
    ).add_to(m)
    x.add_child(folium.features.GeoJsonTooltip(display_vars))
    folium.LayerControl().add_to(m)

    return m

# function for creating data used in nn matching

def create_nn_data(year):
    # create subset
    sub = np.where(years == year)[0][0]
    # create mean values for Vinex locations
    col_names = ['OAD', 'x', 'y']
    # create mean values for Vinex locations
    if year == 2013:
        merged_df = vinex_df[sub].join(vinex_types. \
            set_index('BU2013CODE'),
            on ='statcode',
            how = 'left')
    else: 
        print('Need to figure out how to do this...')
        pass
    vinex_mean_vals = merged_df.loc[:, ['Vinex_naam'] + col_names].groupby('Vinex_naam').mean()
    merged_df = merged_df.join(vinex_mean_vals, on = 'Vinex_naam', rsuffix = '_mean', how = 'left')
    merged_df['Vinex'] = np.where(merged_df['Vinex_naam'].isna(), 0, 1)
    # create conditions for vinex and non vinex
    vinex_conds = (merged_df['Vinex'] == 1) & (merged_df['OAD_mean'].notna())
    non_vinex_conds = (merged_df['Vinex'] == 0) & (merged_df['OAD'].notna())
    # names and arrays with data
    vinex_names = ['Vinex_naam', 'OAD_mean', 'x_mean', 'y_mean']
    non_vinex_names = ['statcode', 'OAD', 'x', 'y']
    vinex_geoms = np.array(merged_df.loc[vinex_conds, vinex_names].drop_duplicates())
    non_vinex_geoms = np.array(merged_df.loc[non_vinex_conds, non_vinex_names].drop_duplicates())
    # names of stadsgewesten 
    sg_conds = (sg_gdf[sub]['statcode'] != 'SG00')
    sg_geoms = np.array(sg_gdf[sub].loc[sg_conds, ['statnaam', 'x', 'y']])
    # update names
    vinex_names = vinex_names + ['dist_mean']
    non_vinex_names = non_vinex_names + ['dist']

    return vinex_names, non_vinex_names, vinex_geoms, non_vinex_geoms, sg_geoms    

In [4]:
# read geometry of buurt indelingen

# download data on gebiedsindelingen if if does not exist
if len(os.listdir(output_path + 'gebiedsindelingen')) == 0:
    read_and_extract_zip('http://download.cbs.nl/overige/cbsgebiedsindelingen-2021-v1.zip')

    
gb_path = output_path + 'gebiedsindelingen/'
search_list = ['buurt', 'gegeneraliseerd'] + [str(year) for year in years]

buurt_gdf = []

# read in the buurt layers for all years
for f in os.listdir(gb_path):
    for layername in fiona.listlayers(gb_path + f):
        if len([item for item in search_list if item in layername]) == 3:
                print('Now reading ' + layername)
                buurt_gdf.append(gpd.read_file(gb_path + f, layer = layername))




Now reading cbs_buurt_2004_gegeneraliseerd
Now reading cbs_buurt_2005_gegeneraliseerd
Now reading cbs_buurt_2006_gegeneraliseerd
Now reading cbs_buurt_2007_gegeneraliseerd
Now reading cbs_buurt_2008_gegeneraliseerd
Now reading cbs_buurt_2009_gegeneraliseerd
Now reading cbs_buurt_2010_gegeneraliseerd
Now reading cbs_buurt_2011_gegeneraliseerd
Now reading cbs_buurt_2012_gegeneraliseerd
Now reading cbs_buurt_2013_gegeneraliseerd


In [17]:

search_list = ['stadsgewest', 'gegeneraliseerd'] + [str(year) for year in years]

sg_gdf = []

# read in the buurt layers for all years
for f in os.listdir(gb_path):
    for layername in fiona.listlayers(gb_path + f):
        if len([item for item in search_list if item in layername]) == 3:
                print('Now reading ' + layername)
                tmp = gpd.read_file(gb_path + f, layer = layername)
                tmp['x'] = tmp.centroid.x
                tmp['y'] = tmp.centroid.y
                sg_gdf.append(tmp)



Now reading cbs_stadsgewest_2004_gegeneraliseerd
Now reading cbs_stadsgewest_2005_gegeneraliseerd
Now reading cbs_stadsgewest_2006_gegeneraliseerd
Now reading cbs_stadsgewest_2007_gegeneraliseerd
Now reading cbs_stadsgewest_2008_gegeneraliseerd
Now reading cbs_stadsgewest_2009_gegeneraliseerd
Now reading cbs_stadsgewest_2010_gegeneraliseerd
Now reading cbs_stadsgewest_2011_gegeneraliseerd
Now reading cbs_stadsgewest_2012_gegeneraliseerd
Now reading cbs_stadsgewest_2013_gegeneraliseerd


In [13]:
## read in buurt and wijk cijfers
bw_df = []

for bw_no in np.arange(len(bw_cijfers_url)):
    bw_df.append(read_and_define_subsample(bw_cijfers_url[bw_no]))


Now downloading version 2004
Now downloading version 2005
Now downloading version 2006
Now downloading version 2007
Now downloading version 2008
Now downloading version 2009
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Now downloading version 2010
Now downloading version 2011
Now downloading version 2012
Now downloading version 2013


In [14]:
# add geometry to data file
vinex_df = []
for idx in np.arange(len(bw_df)):
    vinex_df.append(buurt_gdf[idx].join(bw_df[idx].set_index('statcode'), on = 'statcode'))
    vinex_df[idx]['x'] = vinex_df[idx].centroid.x
    vinex_df[idx]['y'] = vinex_df[idx].centroid.y


In [78]:
vinex_df[0].head()

,statcode,statnaam,jrstatcode,rubriek,geometry,OAD,year,x,y
0,BU00030000,Appingedam-Centrum,2004BU00030000,buurt,"MULTIPOLYGON (((253641.500 594417.812, 253476....",1059,2004.0,253062.354903,594044.583358
1,BU00030001,Appingedam-West,2004BU00030001,buurt,"MULTIPOLYGON (((251430.765 594486.315, 251265....",775,2004.0,251995.900146,593801.869592
2,BU00030002,Appingedam-Oost,2004BU00030002,buurt,"MULTIPOLYGON (((254576.353 594572.311, 253863....",1035,2004.0,253867.594736,593365.696145
3,BU00030007,Verspreide huizen Damsterdiep en Eemskanaal,2004BU00030007,buurt,"MULTIPOLYGON (((249556.069 593213.323, 249389....",159,2004.0,251193.511362,592464.384454
4,BU00030008,Verspreide huizen ten zuiden van Eemskanaal,2004BU00030008,buurt,"MULTIPOLYGON (((254983.407 592487.504, 254823....",61,2004.0,252931.058087,591437.935633


In [24]:

# read in data on Vinex typolog
excel_file_name = 'vinex_2013_indeling/CBS_Buurt2013_Vinex_uitleg_type.xlsx'
vinex_types = pd.read_excel(output_path + excel_file_name)


In [94]:
# find distance to the nearest stadsgewest
vinex_names, non_vinex_names, vinex_geoms, non_vinex_geoms, sg_geoms = create_nn_data(2013)

# set up search tree
tree = KDTree(sg_geoms[:,1:])

# find shortest distance from all vinex 
vinex_dist, vinex_ind = tree.query(vinex_geoms[:,2:], k=1)
vinex_geoms = np.append(vinex_geoms, vinex_dist, 1)

# find shortest distance from all non-vinex
non_vinex_dist, non_vinex_ind = tree.query(non_vinex_geoms[:,2:], k=1)
non_vinex_geoms = np.append(non_vinex_geoms, non_vinex_dist, 1)

In [ ]:
# show Vinex wijken
for idx in vinex_neighbours['Vinex_naam'].drop_duplicates():
    print(idx)

In [112]:
# match on density

# settings
n_neighbours = 20
weight_oad = 0.96
weights = [weight_oad, (1-weight_oad)/2, (1-weight_oad)/2, 0]

# nearest neighbour search
tree = BallTree(non_vinex_geoms[:,1:], metric = 'wminkowski', p = 1, w = weights)
dist, ind = tree.query(vinex_geoms[:,1:], k=n_neighbours)

# gather everything in a data frame
out = []
for idx in np.arange(len(ind)):
    for jdx in np.arange(n_neighbours):
        tmp = np.concatenate((vinex_geoms[idx,],  non_vinex_geoms[ind[idx,],][jdx])).tolist()
        out.append(tmp)

vinex_neighbours = pd.DataFrame(out, columns = vinex_names + non_vinex_names)
#vinex_neighbours.head()

In [113]:
# add matching neighbours into a spatial data frame
vinex_neighbours_gdf = vinex_df[9]. \
         join(vinex_types[['BU2013CODE', 'Vinex_naam']].set_index('BU2013CODE'),
                  on = 'statcode'). \
                       join(vinex_neighbours[['Vinex_naam', 'statcode', 'OAD_mean']]. \
                            set_index('statcode'),
                            on = 'statcode', rsuffix = '_r', how =  'left'). \
                                 to_crs("EPSG:4326")

# clean up columns
vinex_neighbours_gdf['Vinex'] = np.where(vinex_neighbours_gdf['Vinex_naam'].isna(), 0, 1)
vinex_neighbours_gdf['Vinex_naam_r'][vinex_neighbours_gdf['Vinex'] == 1] = vinex_neighbours_gdf['Vinex_naam']
vinex_neighbours_gdf = vinex_neighbours_gdf. \
     rename(columns = {'Vinex_naam_r' : 'Vinex wijk', 'OAD_mean' : 'OAD Vinex wijk'}) 

## mid point of Netherlands
nl_mid = [vinex_neighbours_gdf['geometry'].centroid.y.mean(), vinex_neighbours_gdf['geometry'].centroid.x.mean()] 

## create map for specific Vinex locations

vinex_wijk = ['IJburg', 'De Aker', 'Castellum', 'Leidsche Rijn', 'Nesselande', 'Portland - Carnisselande', 'Plaslocatie - Landscheidingszone', 'Markgraven']
conds = (vinex_neighbours_gdf['Vinex wijk'].isin(vinex_wijk)) | (vinex_neighbours_gdf['Vinex_naam'].isin(vinex_wijk))
display_list = ['statnaam', 'year', 'Vinex wijk', 'OAD', 'OAD Vinex wijk']
m = create_folium_map(vinex_neighbours_gdf[conds], 2,'Vinex', display_list, nl_mid)

m


In [125]:
# match on distance to nearest stadsgewest

# settings
n_neighbours = 20
weight_dist = 0.8
weights = [0, (1-weight_dist)/2, (1-weight_dist)/2, weight_dist]

# nearest neighbour search
tree = BallTree(non_vinex_geoms[:,1:], metric = 'wminkowski', p = 1, w = weights)
dist, ind = tree.query(vinex_geoms[:,1:], k=n_neighbours)

# gather everything in a data frame
out = []
for idx in np.arange(len(ind)):
    for jdx in np.arange(n_neighbours):
        tmp = np.concatenate((vinex_geoms[idx,],  non_vinex_geoms[ind[idx,],][jdx])).tolist()
        out.append(tmp)

vinex_neighbours = pd.DataFrame(out, columns = vinex_names + non_vinex_names)

In [126]:
# add matching neighbours into a spatial data frame
vinex_neighbours_gdf = vinex_df[9]. \
         join(vinex_types[['BU2013CODE', 'Vinex_naam']].set_index('BU2013CODE'),
                  on = 'statcode'). \
                       join(vinex_neighbours[['Vinex_naam', 'statcode', 'OAD_mean']]. \
                            set_index('statcode'),
                            on = 'statcode', rsuffix = '_r', how =  'left'). \
                                 to_crs("EPSG:4326")

# clean up columns
vinex_neighbours_gdf['Vinex'] = np.where(vinex_neighbours_gdf['Vinex_naam'].isna(), 0, 1)
vinex_neighbours_gdf['Vinex_naam_r'][vinex_neighbours_gdf['Vinex'] == 1] = vinex_neighbours_gdf['Vinex_naam']
vinex_neighbours_gdf = vinex_neighbours_gdf. \
     rename(columns = {'Vinex_naam_r' : 'Vinex wijk', 'OAD_mean' : 'OAD Vinex wijk'}) 

## mid point of Netherlands
nl_mid = [vinex_neighbours_gdf['geometry'].centroid.y.mean(), vinex_neighbours_gdf['geometry'].centroid.x.mean()] 

## create map for specific Vinex locations
vinex_wijk = ['IJburg', 'De Aker', 'Castellum', 'Leidsche Rijn', 'Nesselande', 'Portland - Carnisselande', 'Plaslocatie - Landscheidingszone', 'Markgraven']
conds = (vinex_neighbours_gdf['Vinex wijk'].isin(vinex_wijk)) | (vinex_neighbours_gdf['Vinex_naam'].isin(vinex_wijk))
display_list = ['statnaam', 'year', 'Vinex wijk', 'OAD', 'OAD Vinex wijk']
m = create_folium_map(vinex_neighbours_gdf[conds], 2,'Vinex', display_list, nl_mid)

m
